## Создание отчета по HB

In [3]:
import xlsxwriter
import openpyxl
import pandas as pd
import numpy as np
from datetime import datetime as dt
import warnings
warnings.filterwarnings('ignore')
import openpyxl
from openpyxl.styles import Font, PatternFill, Border, Side, Alignment

### Общий отчет

In [4]:
# Load the Excel file into a pandas DataFrame
df_report = pd.read_excel('HB.xlsx', sheet_name='Задание HB', header=1)
df_sales = pd.read_excel('HB.xlsx', sheet_name='Данные_Продажи')
df_sku = pd.read_excel('HB.xlsx', sheet_name='Загрузка_на_борт')

df_report = df_report.drop(['Охват Факт', 'Визиты Факт', 'Факт', 'Факт, %', 'План, %'], axis=1)

# Transform the Artist column in the Sales DataFrame
df_sales['Исполнитель'] = df_sales['Исполнитель'].str.extract(r'([\w ]+\d{2})')
df_sales['Исполнитель'] = df_sales['Исполнитель'].str.lstrip()

# Create a new DataFrame to hold the counts of unique "Point: Code" values
df_counts = df_sales.groupby('Исполнитель')['Точка: Код'].nunique().reset_index()
df_counts.columns = ['Исполнитель', 'Охват Факт']

df_counts_виз = df_sales.groupby('Исполнитель')['Точка: Код'].count().reset_index()
df_counts_виз.columns = ['Исполнитель', 'Визиты Факт']

# Merge the counts DataFrames with the Report DataFrame
df_merged = pd.merge(df_report, df_counts, left_on='Агент', right_on='Исполнитель', how='left')
df_merged = pd.merge(df_merged, df_counts_виз, on='Исполнитель', how='left')

df_merged['Охват Факт'] = pd.to_numeric(df_merged['Охват Факт'], errors='coerce').astype('Int64')
df_merged['Визиты Факт'] = pd.to_numeric(df_merged['Визиты Факт'], errors='coerce').astype('Int64')

# define the custom order of regions
custom_order = ['South-East', 'East', 'Center', 'West', 'North', 'South', 'Grand Total']

# create a categorical data type with the custom order
cat_dtype = pd.CategoricalDtype(categories=custom_order, ordered=True)

# apply the categorical data type to the Region column
df_merged['Регион'] = df_merged['Регион'].astype(cat_dtype)

# sort the dataframe by the Region column
df_merged = df_merged.sort_values('Регион')

df_merged = df_merged.reset_index(drop=True)

In [5]:
# select the columns containing "Total amount" in their name and the "Executor" column
df_sales_sum = df_sales[['Исполнитель'] + list(df_sales.filter(like='Общая сумма').columns)]

# melt the data frame to transform it into long-format
df_sales_sum_melt = df_sales_sum.melt(id_vars='Исполнитель', var_name='Наименование', value_name='Продажи, сумма')

# extract the part before ":" and make it uppercase for the "Наименование" column
df_sales_sum_melt['Наименование'] = df_sales_sum_melt['Наименование'].str.split(':').str[0].str.upper()

# group by executor and name, and sum the sales amount
df_grouped_sum = df_sales_sum_melt.groupby(['Исполнитель', 'Наименование']).sum().reset_index()

# select only the desired columns for the output
df_output_sum = df_grouped_sum[['Исполнитель', 'Наименование', 'Продажи, сумма']]
df_sum = df_output_sum.groupby('Исполнитель')['Продажи, сумма'].sum().reset_index()

df_merged = df_merged.merge(df_sum, on='Исполнитель', how = 'left')

### SKU

In [6]:
# Lowercase all column names in the df_sales dataframe
df_sales = df_sales.rename(columns=dict(zip(df_sales.columns[82:124], [c.split(':')[0].lower() for c in df_sales.columns[82:124]])))
df_sku = df_sales.rename(columns=dict(zip(df_sales.columns[18:39], [c.split(':')[0].lower() for c in df_sales.columns[18:39]])))
sku_names = list(df_sku.columns[18:39])

# Create a new column called "Availability" in the df_sales dataframe and initialize it to 0
df_sales['Наличие'] = 0

# Fill all NaN values in the product columns with False
df_sales[df_sales.columns[82:124]] = df_sales[df_sales.columns[82:124]].fillna("ЛОЖЬ")
df_sales[df_sales.columns[82:124]] = df_sales[df_sales.columns[82:124]].replace({"ЛОЖЬ": 0, "ИСТИНА": 1})

# Loop through each row in the df_sales dataframe
for index, row in df_sales.iterrows():
    # Set the default value for Availability to 1
    availability = 1
    
    # Loop through each product in the products list
    for product in sku_names:
        # Check if the product is in any of the column names in the row and if it is 1 (True)
        if any(product in column and row[column] for column in df_sales.columns):
            continue
        else:
            # If the product is not in any of the column names or is False, set Availability to 0 and break the loop
            availability = 0
            break
    # Set the Availability value for the current row
    df_sales.at[index, 'Наличие'] = availability

In [7]:
df_sales['Наличие'].value_counts()

0    1261
Name: Наличие, dtype: int64

In [8]:
# Initialize the Quantity column to 0
df_sales['Количество'] = 0

# Loop through each row in the df_sales dataframe
for index, row in df_sales.iterrows():
    # Set the default value for Quantity to 0
    quantity = 0
    
    # Loop through each product in the sku_names list
    for product in sku_names:
        # Check if the product name matches any of the column names in the row and if it is 1 (True)
        if any(product in column and row[column]==1 for column in df_sales.columns):
            # If it matches, add 1 to the quantity
            quantity += 1
    
    # Set the Quantity value for the current row
    df_sales.at[index, 'Количество'] = quantity

In [9]:
# Filter df_sales based on availability
df_avail = df_sales[(df_sales['Количество'] >= 16) & (df_sales['Количество'] < 21)]

# Group by Artist and Point: Code
df_group_avail = df_avail.groupby(['Исполнитель', 'Точка: Код']).agg({'Количество': 'sum'})

# Define a function to find missing items
def find_missing_items(row):
    missing_items = []
    for product in sku_names:
        if product not in row.index:
            missing_items.append(product)
    return missing_items

# Apply the find_missing_items function to create a new column with missing items
df_group_avail['Отсутствующие SKU'] = df_group_avail.apply(find_missing_items, axis=1)

# Reset the index
df_group_avail = df_group_avail.reset_index()

# Select the desired columns
df_result = df_group_avail[['Исполнитель', 'Точка: Код', 'Отсутствующие SKU']]

In [11]:
df_count = df_sales[df_sales['Наличие'] == 0].drop_duplicates(subset=['Исполнитель',
                                                                      'Точка: Код']).groupby('Исполнитель').agg({'Точка: Код': 'count'}).reset_index()
df_count = df_count.rename(columns={'Точка: Код': 'Кол-во ТТ c отсут. SKU'})
df_merged = df_merged.merge(df_count, on='Исполнитель', how='left')
df_merged = df_merged.drop('Исполнитель', axis=1)

,Регион,Дистрикт,Агент,Охват План,Визиты План,План IMS,План,Охват Факт,Визиты Факт,"Продажи, сумма",Кол-во ТТ c отсут. SKU
0,South-East,Taldykorgan Dst.,Taldyk HB 01,120,300,NaN,36.0,52,107,3443924.0,52.0
1,South-East,Almaty Dst.,Almaty HB 01,100,300,NaN,30.0,20,28,969147.0,20.0
2,South-East,Almaty Dst.,Almaty HB 02,100,300,NaN,30.0,<NA>,<NA>,NaN,NaN
3,East,Oskemen Dst.,OSM HB 01,76,300,NaN,38.0,33,62,1462632.0,33.0
4,East,Oskemen Dst.,OSM HB 02,48,300,NaN,24.0,<NA>,<NA>,NaN,NaN


In [13]:
# Filter rows with Availability equals 1
df_available = df_sales.loc[df_sales['Наличие'] == 1]

# Create new dataframe with only Point: Code and Availability columns
df_point_availability = df_available.loc[:, ['Точка: Код', "Точка: Адрес", "Исполнитель", 'Наличие']].drop_duplicates()

In [14]:
df_merged = df_merged.fillna(0)
df_sales_r = df_sales[df_sales['Наличие'] == 1].drop_duplicates(subset=['Исполнитель', 'Точка: Код'], keep='first')
df_fact = df_sales_r.groupby('Исполнитель')['Наличие'].count().reset_index(name='Факт')
df_merged = pd.merge(df_merged, df_fact, left_on='Агент', right_on='Исполнитель', how='left')
df_merged['Факт, %'] = np.round((pd.to_numeric(df_merged['Факт'])/df_merged['Охват План']), 2)
df_merged['План'] = np.round(df_merged['План'], 0)
df_merged['План, %'] = np.round((df_merged['План']/df_merged['Охват План']), 2)
df_merged['Факт'] = pd.to_numeric(df_merged['Факт'], errors='coerce').astype('Int64')
df_merged = df_merged.fillna(0)

df_merged['Охват Разница'] = df_merged['Охват План'] - df_merged['Охват Факт']
df_merged['Визиты Разница'] = df_merged['Визиты План'] - df_merged['Визиты Факт']
df_merged['Продажи Разница'] = df_merged['План IMS'] - df_merged['Продажи, сумма']
df_merged['Наличие Разница'] = df_merged['Факт'] - df_merged['План']

df_merged = df_merged.reindex(columns=['Регион', 'Дистрикт', 'Агент', 'Охват План', 'Охват Факт', 'Охват Разница',
                                       'Визиты План', 'Визиты Факт',  'Визиты Разница', 'План IMS', 'Продажи, сумма',
                                       'Продажи Разница',  'План', 'План, %', 'Факт', 'Факт, %', 'Наличие Разница',
                                       'Кол-во ТТ c отсут. SKU'])
df_merged.head()

,Регион,Дистрикт,Агент,Охват План,Охват Факт,Охват Разница,Визиты План,Визиты Факт,Визиты Разница,План IMS,"Продажи, сумма",Продажи Разница,План,"План, %",Факт,"Факт, %",Наличие Разница,Кол-во ТТ c отсут. SKU
0,South-East,Taldykorgan Dst.,Taldyk HB 01,120,52,68,300,107,193,0.0,3443924.0,-3443924.0,36.0,0.3,0,0.0,-36.0,52.0
1,South-East,Almaty Dst.,Almaty HB 01,100,20,80,300,28,272,0.0,969147.0,-969147.0,30.0,0.3,0,0.0,-30.0,20.0
2,South-East,Almaty Dst.,Almaty HB 02,100,0,100,300,0,300,0.0,0.0,0.0,30.0,0.3,0,0.0,-30.0,0.0
3,East,Oskemen Dst.,OSM HB 01,76,33,43,300,62,238,0.0,1462632.0,-1462632.0,38.0,0.5,0,0.0,-38.0,33.0
4,East,Oskemen Dst.,OSM HB 02,48,0,48,300,0,300,0.0,0.0,0.0,24.0,0.5,0,0.0,-24.0,0.0


In [15]:
# Select numeric columns except those with a "%" in the column name
numeric_cols = df_merged.select_dtypes(include=['number']).columns.tolist()
numeric_cols = [col for col in numeric_cols if ("%" not in col) and ("Разница" not in col)]

# Compute the sum for each selected column in the bottom row
sums = df_merged[numeric_cols].sum(numeric_only=True, min_count=1)
sums.name = "Total"

# Append the sums as a new row to the bottom of the dataframe
df_merged = df_merged.append(sums)

### Отчет по продажам

In [16]:
# read the necessary data from Excel sheets
df_sales = pd.read_excel('HB.xlsx', sheet_name='Данные_Продажи')
df_remaining = pd.read_excel('HB.xlsx', sheet_name='Остатки_Январь')
df_loading = pd.read_excel('HB.xlsx', sheet_name='Загрузка_на_борт')
df_agent = pd.read_excel('HB.xlsx', sheet_name='Данные_по_агентам')
df_sku_data = pd.read_excel('HB.xlsx', sheet_name='Данные_по_SKU')

# rename the SKU columns in the 'df_remaining' data frame
df_remaining = df_remaining.rename(columns=dict(zip(df_remaining.columns[2:23],
                                                    [c.split(':')[0].lower() for c in df_remaining.columns[2:23]])))
df_sales['Исполнитель'] = df_sales['Исполнитель'].str.strip()
df_remaining['Исполнитель'] = df_remaining['Исполнитель'].str.strip()

# melt the data frame to get the SKU and quantity in separate rows
df_melted = pd.melt(df_remaining, id_vars=['Исполнитель'], value_vars=df_remaining.columns[2:],
                    var_name='Наименование', value_name='Приход')

# extract the part before ":" and make it uppercase for the "Наименование" column
df_melted['Наименование'] = df_melted['Наименование'].str.split(':').str[0].str.upper()

# sort the melted data frame by executor and name
df_melted_sort = df_melted.sort_values(by=['Исполнитель', 'Наименование'])
#df_melted_sort['Исполнитель'] = df_melted_sort['Исполнитель'].str.extract(r'([\w ]+\d{2})')

# select the columns containing "Quantity" in their name and the "Executor" column
df_sales_qty = df_sales[['Исполнитель'] + list(df_sales.filter(like='Количество').columns)]

# melt the data frame to transform it into long-format
df_sales_qty_long = df_sales_qty.melt(id_vars='Исполнитель', var_name='Наименование',
                                      value_name='Продажи, количество')

# extract the part before ":" and make it uppercase for the "Наименование" column
df_sales_qty_long['Наименование'] = df_sales_qty_long['Наименование'].str.split(':').str[0].str.upper()

# group by executor and name, and sum the sales quantity
df_grouped = df_sales_qty_long.groupby(['Исполнитель', 'Наименование']).sum().reset_index()

# select only the desired columns for the output
df_output = df_grouped[['Исполнитель', 'Наименование', 'Продажи, количество']]

# select the columns containing "Total amount" in their name and the "Executor" column
df_sales_sum = df_sales[['Исполнитель'] + list(df_sales.filter(like='Общая сумма').columns)]

# melt the data frame to transform it into long-format
df_sales_sum_melt = df_sales_sum.melt(id_vars='Исполнитель', var_name='Наименование', value_name='Продажи, сумма')

# extract the part before ":" and make it uppercase for the "Наименование" column
df_sales_sum_melt['Наименование'] = df_sales_sum_melt['Наименование'].str.split(':').str[0].str.upper()

# group by executor and name, and sum the sales amount
df_grouped_sum = df_sales_sum_melt.groupby(['Исполнитель', 'Наименование']).sum().reset_index()

# select only the desired columns for the output
df_output_sum = df_grouped_sum[['Исполнитель', 'Наименование', 'Продажи, сумма']]

df_price = df_sales[['Исполнитель'] + list(df_sales.filter(like='Цена').columns)]
df_price = df_price.melt(id_vars='Исполнитель', var_name='Наименование', value_name='Цена уст.')
df_price['Наименование'] = df_price['Наименование'].str.split(':').str[0].str.upper()
df_price_group = df_price.groupby(['Исполнитель', 'Наименование']).first().reset_index()
df_price_group = df_price_group[['Исполнитель', 'Наименование', 'Цена уст.']]

df_loading = df_loading.rename(columns=dict(zip(df_loading.columns[18:39],
                                                [c.split(':')[0].upper() for c in df_loading.columns[18:39]])))
product_columns = [col for col in df_loading.columns[18:39]]

df_grouped_load = df_loading.groupby('Исполнитель')[product_columns].sum().reset_index()

# melt the data frame to transform it into long-format
df_melted_load = df_grouped_load.melt(id_vars='Исполнитель', value_vars=product_columns,
                                      var_name='Наименование', value_name='Загрузка')

# sort the data frame by artist and name
df_out_load = df_melted_load.sort_values(['Исполнитель', 'Наименование']).reset_index(drop=True)

df_out_ = df_melted_sort.merge(df_output, on=['Исполнитель', 'Наименование'], how = 'left')
df_out_ = df_out_.merge(df_output_sum, on=['Исполнитель', 'Наименование'], how = 'left')
df_out_ = df_out_.merge(df_price_group, on=['Исполнитель', 'Наименование'], how = 'left')
df_out_ = df_out_.merge(df_out_load, on=['Исполнитель', 'Наименование'], how = 'left')

df_out_['Продажи, количество'] = df_out_['Продажи, количество'].fillna(0).astype('int64')
df_out_['Продажи, сумма'] = np.round(df_out_['Продажи, сумма'], 2)
# add the new columns
df_out_['Цена за ед.'] = np.round((df_out_['Продажи, сумма'] / df_out_['Продажи, количество']), 1)
df_out_['Проверка цены'] = np.round((df_out_['Цена за ед.'] - df_out_['Цена уст.']), 1)
# Merge the two dataframes on the 'Artist' and 'SKU' columns
df_out_ = df_out_.merge(df_sku_data[['Наименование', 'Маржа агентства']], on=['Наименование'], how='left')
df_out_['Доход агентства'] = np.round((df_out_['Продажи, сумма'] * df_out_['Маржа агентства']), 2)
df_out_['Остатки'] = (df_out_['Приход'] + df_out_['Загрузка'] - df_out_['Продажи, количество']).fillna(0).astype('int64')
df_out_['Стоимость остатков'] = np.round((df_out_['Остатки'] * df_out_['Цена за ед.']), 2)
#df_out_ = df_out_.drop('Загрузка', axis=1)
df_out_.reset_index(drop=True)
leftovers = df_out_.groupby('Исполнитель')['Стоимость остатков'].sum().reset_index()
df_agent = df_agent.merge(leftovers, on=['Исполнитель'], how='left')
df_agent['Долг'] = df_agent['Инкасация'] - df_agent['Стоимость остатков']

In [18]:
df_out_['Исполнитель'].value_counts()

Aktau HB 01 Избасов Б.М              21
Kokshetau HB 02 Байбол Е.Қ           21
Turkestan HB 01 Кәрімов С.Б.         21
Taldyk HB 01 Нарманов М.А.           21
Semey HB 01 Тулеужанов А.            21
OSM HB 01 Кенжегаринов Э.            21
Kyzylorda HB 01 Жылкайдар А.М.       21
Kostanay HB 01 Байменов А.           21
Kokshetau HB 01 Корниенко В.В.       21
Aktobe HB 01 Тасбулатов Р.М.         21
Karaganda HB 01 Абдрахманов Д. Е.    21
Atyrau HB 01 Еснгельдиев. А          21
Atbasar HB 01 Ковальчук Р. В.        21
Almaty HB 01 Орынбасар Д. А          21
Aktobe HB 03 Тулегенов А.М.          21
Aktobe HB 02 Жиенғалиев И.К.         21
Turkestan HB 02 Толеген Б.Б          21
Name: Исполнитель, dtype: int64

In [ ]:
df_agent['Исполнитель'].value_counts()

In [ ]:
df_sales['Исполнитель'].value_counts()

In [21]:
# Save the changes to the Excel file
with pd.ExcelWriter('HB_1.xlsx') as writer:
    df_merged.to_excel(writer, sheet_name='Задание HB', index=False)
    df_point_availability.to_excel(writer, sheet_name='Наличие по 21 SKU', index=False)
    df_out_.to_excel(writer, sheet_name='Отчет_по_продажам', index=False)
    df_agent.to_excel(writer, sheet_name='Данные_по_агентам', index=False)
    df_sku_data.to_excel(writer, sheet_name='Данные_по_SKU', index=False)
    df_sales.to_excel(writer, sheet_name='Данные_Продажи', index=False)
    df_loading.to_excel(writer, sheet_name='Загрузка_на_борт', index=False)
    df_remaining.to_excel(writer, sheet_name='Остатки', index=False)

In [22]:
# Load the Excel file
workbook = openpyxl.load_workbook('HB_1.xlsx')

# Select the sheet you want to format
sheet1 = workbook['Задание HB']
sheet2 = workbook['Отчет_по_продажам']
sheet3 = workbook['Данные_по_агентам']
sheet4 = workbook['Данные_по_SKU']
sheet5 = workbook['Остатки']
sheet6 =  workbook['Наличие по 21 SKU']

for sheet in [sheet1, sheet2, sheet3, sheet4, sheet5, sheet6]:
    sheet.column_dimensions['A'].width = 30
    sheet.column_dimensions['B'].width = 28
    # Define the column widths
    column_widths = [15] * 16
    # Set the width of the columns
    for i, width in enumerate(column_widths):
        column_letter = chr(ord('C') + i)
        sheet.column_dimensions[column_letter].width = width
        # Set the height of the first row to 30 (adjust as needed)
        sheet.row_dimensions[1].height = 36
        # Set the font for the column headings
    font = Font(bold=True)
    for col in range(1, sheet.max_column+1):
        cell = sheet.cell(row=1, column=col)
        cell.font = font
    # Set the fill color for the column headings
    fill = PatternFill(start_color='DDDDDD', end_color='DDDDDD', fill_type='solid')
    for col in range(1, sheet.max_column+1):
        cell = sheet.cell(row=1, column=col)
        cell.fill = fill
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    # Set the border for all cells in the sheet
    border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    for row in range(1, sheet.max_row+1):
        for col in range(1, sheet.max_column+1):
            cell = sheet.cell(row=row, column=col)
            cell.border = border
    # Set the fill color for the data rows
    alignment = Alignment(vertical='center')
    fill = PatternFill(start_color='FFFFFF', end_color='FFFFFF', fill_type='solid')
    for row in range(2, sheet.max_row+1):
        for col in range(1, sheet.max_column+1):
            cell = sheet.cell(row=row, column=col)
            cell.fill = fill 
            cell.alignment = alignment

# Save the updated Excel file
workbook.save('Отчет по HB.xlsx')

print('\n'+'Файл сохранён под именем: Отчет по HB')


Файл сохранён под именем: Отчет по HB


### Общая таблица

In [23]:
df_merged

,Регион,Дистрикт,Агент,Охват План,Охват Факт,Охват Разница,Визиты План,Визиты Факт,Визиты Разница,План IMS,"Продажи, сумма",Продажи Разница,План,"План, %",Факт,"Факт, %",Наличие Разница,Кол-во ТТ c отсут. SKU
0,South-East,Taldykorgan Dst.,Taldyk HB 01,120.0,52.0,68,300.0,107.0,193,0.0,3443924.0,-3443924.0,36.0,0.30,0.0,0.0,-36.0,52.0
1,South-East,Almaty Dst.,Almaty HB 01,100.0,20.0,80,300.0,28.0,272,0.0,969147.0,-969147.0,30.0,0.30,0.0,0.0,-30.0,20.0
2,South-East,Almaty Dst.,Almaty HB 02,100.0,0.0,100,300.0,0.0,300,0.0,0.0,0.0,30.0,0.30,0.0,0.0,-30.0,0.0
3,East,Oskemen Dst.,OSM HB 01,76.0,33.0,43,300.0,62.0,238,0.0,1462632.0,-1462632.0,38.0,0.50,0.0,0.0,-38.0,33.0
4,East,Oskemen Dst.,OSM HB 02,48.0,0.0,48,300.0,0.0,300,0.0,0.0,0.0,24.0,0.50,0.0,0.0,-24.0,0.0
5,East,Semey Dst.,Semey HB 01,62.0,60.0,2,300.0,137.0,163,0.0,5972429.0,-5972429.0,31.0,0.50,0.0,0.0,-31.0,60.0
6,Center,Karaganda Dst.,Karaganda HB 01,64.0,13.0,51,300.0,21.0,279,0.0,1112590.0,-1112590.0,32.0,0.50,0.0,0.0,-32.0,13.0
7,Center,Karaganda Dst.,Karaganda HB 03,45.0,0.0,45,300.0,0.0,300,0.0,0.0,0.0,22.0,0.49,0.0,0.0,-22.0,0.0
8,Center,Astana Dst.,Atbasar HB 01,104.0,19.0,85,300.0,26.0,274,0.0,1038824.4,-1038824.4,52.0,0.50,0.0,0.0,-52.0,19.0
9,West,Atyrau Dst.,Aktau HB 02,43.0,0.0,43,300.0,0.0,300,0.0,0.0,0.0,17.0,0.40,0.0,0.0,-17.0,0.0


### Таблица по продажам

In [24]:
df_out_

,Исполнитель,Наименование,Приход,"Продажи, количество","Продажи, сумма",Цена уст.,Загрузка,Цена за ед.,Проверка цены,Маржа агентства,Доход агентства,Остатки,Стоимость остатков
0,Aktau HB 01 Избасов Б.М,HEETS AMBER,NaN,33,22185.2,660.4,80.0,672.3,11.9,0.0164,363.84,0,0.0
1,Aktau HB 01 Избасов Б.М,HEETS BRONZE,NaN,42,28296.8,660.4,80.0,673.7,13.3,0.0164,464.07,0,0.0
2,Aktau HB 01 Избасов Б.М,HEETS GREEN ZING,NaN,83,56297.2,660.4,100.0,678.3,17.9,0.0164,923.27,0,0.0
3,Aktau HB 01 Избасов Б.М,HEETS PURPLE,NaN,163,110445.2,660.4,180.0,677.6,17.2,0.0164,1811.30,0,0.0
4,Aktau HB 01 Избасов Б.М,HEETS SILVER,NaN,34,22929.6,660.4,80.0,674.4,14.0,0.0164,376.05,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,Turkestan HB 02 Толеген Б.Б,PARLIAMENT RESERVE,NaN,80,57424.0,717.8,250.0,717.8,0.0,0.0156,895.81,0,0.0
353,Turkestan HB 02 Толеген Б.Б,PARLIAMENT SILVER BLUE,NaN,90,67122.0,745.8,450.0,745.8,0.0,0.0150,1006.83,0,0.0
354,Turkestan HB 02 Толеген Б.Б,PARLIAMENT WINTER FUSE,NaN,40,28712.0,717.8,150.0,717.8,0.0,0.0156,447.91,0,0.0
355,Turkestan HB 02 Толеген Б.Б,PHILIP MORRIS COMPACT BLUE,NaN,10,6440.0,644.0,150.0,644.0,0.0,0.0159,102.40,0,0.0


### Таблица по долгам

In [25]:
df_agent

,Исполнитель,Инкасация,Стоимость остатков,Долг
0,Almaty HB 01 Орынбасар Д. А,0,0.0,0.0
1,Kokshetau HB 02 Байбол Е.Қ,0,0.0,0.0
2,Aktobe HB 03 Тулегенов А.М.,0,0.0,0.0
3,Aktobe HB 01 Тасбулатов Р.М.,0,0.0,0.0
4,Aktobe HB 02 Жиенғалиев И.К.,0,0.0,0.0
5,Semey HB 01 Тулеужанов А.,0,0.0,0.0
6,Taldyk HB 01 Нарманов М.А.,0,0.0,0.0
7,Karaganda HB 01 Абдрахманов Д. Е.,0,0.0,0.0
8,Atbasar HB 01 Ковальчук Р. В.,0,0.0,0.0
9,Turkestan HB 01 Кәрімов С.Б.,0,0.0,0.0


In [2]:
df_out_['Исполнитель'].value_counts()

NameError: name 'df_out_' is not defined

In [ ]:
df_merged = 

In [ ]:
df_atyrau = df_out_.loc[df_out_['Исполнитель' == '']]